# Using FeatureExtraction

Example followed: https://ohdsi.github.io/FeatureExtraction/articles/UsingFeatureExtraction.html

by Anja van Gestel

In [ ]:
## in R:
# remotes::install_github("OHDSI/FeatureExtraction")

%load_ext autoreload
%autoreload 2

import os
os.environ["R_HOME"] = r"C:\Program Files\R\R-4.3.1" # change as needed

from ohdsi import feature_extraction
from ohdsi import common
from ohdsi.database_connector import Connect, Sql
from ohdsi.sqlrender import RenderSql, HelperFunctions as HelperFunctions

# Introduction

The <code>FeatureExtraction</code> package can be used to create features for a cohort, using the information stored in the Common Data Model. A cohort is defined a set of persons who satisfy one or more inclusion criteria for a duration of time. Features can for example be diagnoses observed prior to entering the cohort. Some people might also refer to such features as ‘baseline characteristics’, or to features in general as ‘covariates’, and we will use those terms interchangeably throughout this vignette.

This vignette describes how features can be constructed using the default covariate definitions embedded in the package. Although these definitions allow quite some customization through predefined parameters, it is possible that someone needs more customization. In this case, the reader is referred to the other vignettes included in this package that deal with constructing completely custom covariates.

This vignette will first describe how to specify which features to construct. In many situations, for example when using <code>FeatureExtraction</code> as part of another package such as <code>CohortMethod</code> or <code>PatientLevelPrediction</code>, that is all one needs to know about the <code>FeatureExtraction</code> package, as the actual calling of the package is done by the other package. However, it is also possible to use this package on its own, for example to create a descriptive characterization of a cohort to include in a paper.

# Covariate settings

Users can specify which covariates to construct in three ways:

1. Choose the default set of covariates.
2. Choose from a set of prespecified analyses.
3. Create a set of custom analyses.

An <b>analysis</b> is a process that creates one or more similar covariates. For example, one analysis might create a binary covariate for each condition observed in the condition_occurrence table in the year prior to cohort start, and another analysis might create a single covariate representing the Charlson comorbidity index.

Note that it is always possible to specify a set of concept IDs that can or can’t be used to construct features. When choosing the default set (option 1) or the prespecified analysis (option 2) this can only be done across all analysis. When creating custom analyses (option 3) this can be specified per analysis.

For <b>advanced users</b>: It is also possible to specify a set of covariate IDs that need to be constructed. A covariate ID identifies a specific covariate, for example the Charlson comorbidity index, or the occurrence of a specific condition concept in a specific time window. A covariate ID is therefore not to be confused with a concept ID. The typical scenario where one might want to specify covariate IDs to construct is when someone already constructed covariates in one population, found a subset of covariates to be of interest, and would like to have only those covariates constructed in another population.

## Using the default set of covariates

Using the default set of covariates is straightforward:

In [ ]:
settings = feature_extraction.create_default_covariate_settings()

This will create a wide array of features, ranging from demographics, through conditions and drugs, to several risk scores.

Note that we could specify a set of concepts that should not be used to create covariates, for example:

In [ ]:
settings = feature_extraction.create_default_covariate_settings(
    excluded_covariate_concept_ids = [1124300],
    add_descendants_to_exclude = True)

This will create the default set of covariates, except those derived from concept 1124300 (the ingredient diclofenac) and any of its descendants (ie. all drugs containing the ingredient diclofenac).

## Using prespecified analyses

The function <code>createCovariateSettings</code> allow the user to choose from a large set of predefined covariates. Type <code>?createCovariateSettings</code> to get an overview of the available options. For example:

In [ ]:
settings = feature_extraction.create_covariate_settings(
    use_demographics_gender = True,
    use_demographics_age_group = True,
    use_condition_occurrence_any_time_prior = True)

This will create binary covariates for gender, age (in 5 year age groups), and each concept observed in the <code>condition_occurrence</code> table any time prior to (and including) the cohort start date.

Many of the prespecified analyses refer to a short, medium, or long term time window. By default, these windows are defined as:

- Long term: 365 days prior up to and including the cohort start date.
- Medium term: 180 days prior up to and including the cohort start date.
- Short term: 30 days prior up to and including the cohort start date.

However, the user can change these values. For example:

In [ ]:
settings = feature_extraction.create_covariate_settings(
    use_condition_era_long_term = True,
    use_condition_era_short_term = True,
    use_drug_era_long_term = True,
    use_drug_era_short_term = True,
    long_term_start_days = -180,
    short_term_start_days = -14,
    end_days = -1)

This redefines the long term window as 180 days prior up to (but not including) the cohort start date, and redefines the short term window as 14 days prior up to (but not including) the cohort start date.

Again, we can also specify which concept IDs should or should not be used to construct covariates:

In [ ]:
settings = feature_extraction.create_covariate_settings(
    use_condition_era_long_term = True,
    use_condition_era_short_term = True,
    use_drug_era_long_term = True,
    use_drug_era_short_term = True,
    long_term_start_days = -180,
    short_term_start_days = -14,
    end_days = -1,
    excluded_covariate_concept_ids = 1124300,
    add_descendants_to_exclude = True)

## Creating a set of custom covariates

This option should only be used by <b>advanced users</b>. It requires one to understand that at the implementation level, an analysis is a combination of a piece of highly parameterized SQL together with a specification of the parameter values. The best way to understand the available options is to take a prespecified analysis as starting point, and convert it to a detailed setting object:

In [ ]:
settings = feature_extraction.create_covariate_settings(use_condition_era_long_term = True)
settings2 = feature_extraction.convert_prespec_settings_to_detailed_settings(settings)

settings2.analyses

One can create a detailed analysis settings object from scratch, and use it to create a detailed settings object:

In [ ]:
analysis_details = feature_extraction.create_analysis_details(
    analysis_id = 1,
    sql_file_name = r"path_to\DemographicsGender.sql",
    parameters = {
        'analysis_id': 1,
        'analysis_name': "Gender",
        'domain_id': "Demographics"},
    included_covariate_concept_ids = [],
    add_descendants_to_include = False,
    excluded_covariate_concept_ids = [],
    add_descendants_to_exclude = False,
    included_covariate_ids = [])

settings = feature_extraction.create_detailed_covariate_settings(list(analysis_details))

## Temporal covariates

Ordinarily, covariates are created for just a few time windows of interest, for example the short, medium, and long term windows described earlier. However, sometimes a more fine-grained temporal resolution is required, for example creating covariates for each day separately, in the 365 days prior to cohort start. We will refer to this type of covariates as temporal covariates. Temporal covariates share the same covariate ID across the time windows, and use a separate time ID to distinguish between time windows. There currently aren’t many applications that are able to handle temporal covariates. For example, the CohortMethod package will break when provided with temporal covariates. However, there are some machine learning algorithms in the PatientLevelPrediction package that require temporal covariates.

Again, we can just choose to use the default settings:

In [ ]:
settings = feature_extraction.create_default_temporal_covariate_settings()

Or, we can choose from a set of prespecified temporal covariates:

In [ ]:
settings2 = feature_extraction.create_temporal_covariate_settings(
    use_condition_occurrence = True,
    use_measurement_value = True)

In this case we’ve chosen to create binary covariates for each concept in the <code>condition_occurrence</code> table, and continuous covariates for each measurement - unit combination in the <code>measurement</code> table in the CDM. By default, temporal covariates are created for each day separately in the 365 days before (but not including) the cohort start date. Different time windows can also be specified, for example creating 7 day intervals instead:

In [ ]:
settings = feature_extraction.create_temporal_covariate_settings(
    use_condition_occurrence = True,
    use_measurement_value = True,
    temporal_start_days = [-364, -7, 7],
    temporal_end_days = [-358, -1, 7])

Each time window includes the specified start and end day.

Similar to ordinary covariates, <b>advanced users</b> can also define custom analyses:

In [ ]:
analysis_details = feature_extraction.create_analysis_details(
    analysis_id = 1,
    sql_file_name = r"path_to\MeasurementValue.sql",
    parameters = {
        'analysis_id': 1,
        'analysis_name': "MeasurementValue",
        'domainId': "Measurement"},
    included_covariate_concept_ids = [],
    add_descendants_to_include = False,
    excluded_covariate_concept_ids = [],
    add_descendants_to_exclude = False,
    included_covariate_ids = [])

settings = feature_extraction.create_detailed_temporal_covariate_settings(list(analysis_details))

### Temporal sequence covariates

You can also generate temporal sequence covariate settings with the following function:

In [ ]:
settings = feature_extraction.create_temporal_sequence_covariate_settings(
    use_demographics_gender = True,
    use_demographics_age = False,
    use_demographics_age_group = True,
    use_demographics_race = True,
    use_demographics_ethnicity = True,
    use_demographics_index_year = True,
    use_demographics_index_month = True,
    use_condition_occurrence = False,
    use_condition_occurrence_primary_inpatient = False,
    use_condition_era_start = False,
    use_condition_era_group_start = False,
    use_drug_exposure = False,
    use_drug_era_start = False,
    use_drug_era_group_start = False,
    use_procedure_occurrence = True,
    use_device_exposure = True,
    use_measurement = True,
    use_measurement_value = False,
    use_observation = True,
    time_part = "DAY",
    time_interval = 1,
    sequence_end_day = -1,
    sequence_start_day = -730,
    included_covariate_concept_ids = [],
    add_descendants_to_include = False,
    excluded_covariate_concept_ids = [],
    add_descendants_to_exclude = False,
    included_covariate_ids = []
)

## Investigating the covariate data object

Some usefull functions when working with covariate data objects:
- <code>create_empty_covariate_data</code> creates an empty covariate data object.
- <code>is_covariate_data</code> checks if an object is a covariate data object.
- <code>is_aggregated_covariate_data</code> checks if a covariate data object is aggregated.
- <code>is_temporal_covariate_data</code> checks if a covariate data object is temporal.

In the cell below we first create a covariate data object for which you can play around with the arguments <code>aggregated</code> and <code>temporal</code>.

In [ ]:
cov_data_tmp = feature_extraction.create_empty_covariate_data(
    cohort_id = 1, 
    aggregated = False, 
    temporal = True
)

is_covariate_data = feature_extraction.is_covariate_data(cov_data_tmp)
is_aggregated_covariate_data = feature_extraction.is_aggregated_covariate_data(cov_data_tmp)
is_temporal_covariate_data = feature_extraction.is_temporal_covariate_data(cov_data_tmp)

print(f'is_covariate_data: {is_covariate_data}')
print(f'is_aggregated_covariate_data: {is_aggregated_covariate_data}')
print(f'is_temporal_covariate_data: {is_temporal_covariate_data}')

# Constructing covariates for a cohort of interest

Here we will walk through an example, creating covariates for two cohorts of interest: <br>
new users of etonogestrel (1519936) and new users of paclitaxel (1378382)

## Configuring the connection to the server

We need to tell R how to connect to the server where the data are. <code>CohortMethod</code> uses the <code>DatabaseConnector</code> package, which provides the <code>createConnectionDetails</code> function. Type <code>?createConnectionDetails</code> for the specific settings required for the various database management systems (DBMS). For example, one might connect to a PostgreSQL database using this code:

In [ ]:
connection_details = Connect.create_connection_details(
    dbms="postgresql",
    server="localhost/cdm_database",
    user="postgres",
    password="password",
    port=5432)
conn = Connect.connect(connection_details)

cdm_database_schema = "cdm_schema"
results_database_schema = "results"

The last two lines define the <code>cdmDatabaseSchema</code> and <code>resultSchema</code> variables. We’ll use these later to tell R where the data in CDM format live, and where we want to write intermediate and result tables. Note that for Microsoft SQL Server, database schemas need to specify both the database and the schema, so for example <code>cdmDatabaseSchema <- "my_cdm_data.dbo"</code>.

## Creating a cohort of interest

FeatureExtraction requires the cohorts to be instantiated in the <code>cohort</code> table in the Common Data Model, or in a table that has the same structure as the <code>cohort</code> table. We could create cohorts using a cohort definition tool, but here we’ll just use some simple SQL to find the first drug era per person. Note that because we will be creating covariates based on data before cohort start, we are requiring 365 days of observation before the first exposure. FeatureExtraction will not check if a subject is observed during the specified time windows.

```sql
/***********************************
File cohorts_of_interest.sql
***********************************/

IF OBJECT_ID('@resultsDatabaseSchema.cohorts_of_interest', 'U') IS NOT NULL
DROP TABLE @resultsDatabaseSchema.cohorts_of_interest;

SELECT first_use.*
INTO @resultsDatabaseSchema.cohorts_of_interest
FROM (
SELECT drug_concept_id AS cohort_definition_id,
MIN(drug_era_start_date) AS cohort_start_date,
MIN(drug_era_end_date) AS cohort_end_date,
person_id AS subject_id
FROM @cdmDatabaseSchema.drug_era
WHERE drug_concept_id = 1118084-- celecoxib
OR drug_concept_id = 1124300 --diclofenac
GROUP BY drug_concept_id, 
person_id
) first_use 
INNER JOIN @cdmDatabaseSchema.observation_period
ON first_use.subject_id = observation_period.person_id
AND cohort_start_date >= observation_period_start_date
AND cohort_end_date <= observation_period_end_date
WHERE DATEDIFF(DAY, observation_period_start_date, cohort_start_date) >= 365;
```

This is parameterized SQL which can be used by the SqlRender package. We use parameterized SQL so we do not have to pre-specify the names of the CDM and result schemas. That way, if we want to run the SQL on a different schema, we only need to change the parameter values; we do not have to change the SQL code. By also making use of translation functionality in SqlRender, we can make sure the SQL code can be run in many different environments.

<b>NOTE</b><br>
In the example online the SQL was not completely correct I changed the following lines:
- person_id --> person_id AS subject_id
- ON first_use.person_id = observation_period.person_id --> ON first_use.subject_id = observation_period.person_id

In [ ]:
sql = HelperFunctions.read_sql(r"path_to\cohorts_of_interest.sql")
sql = RenderSql.render(
    sql,
    cdmDatabaseSchema = cdm_database_schema,
    resultsDatabaseSchema = results_database_schema)
sql = RenderSql.translate(sql, 'postgresql', temp_emulation_schema=None)

Sql.execute_sql(conn, sql)

In this code, we first read the SQL from the file into memory. In the next line, we replace the two parameter names with the actual values. We then translate the SQL into the dialect appropriate for the DBMS we already specified in the <code>connectionDetails</code>. Next, we connect to the server, and submit the rendered and translated SQL.

If all went well, we now have a table with the cohorts of interest. We can see how many events per type:

In [ ]:
sql = "SELECT cohort_definition_id, COUNT(*) AS count FROM @resultsDatabaseSchema.cohorts_of_interest GROUP BY cohort_definition_id"
sql = RenderSql.render(sql, resultsDatabaseSchema = results_database_schema)
sql = RenderSql.translate(sql, 'postgresql', temp_emulation_schema=None)

result = Sql.query_sql(conn, sql)
common.convert_from_r(result)

## Creating default covariates

Or create a large default set of covariates, including covariates for all drugs, drug classes, condition, condition classes, procedures, observations, etc.

In [ ]:
covariate_settings = feature_extraction.create_default_covariate_settings()

covariate_data = feature_extraction.get_db_default_covariate_data(
    cdm_database_schema = cdm_database_schema,
    connection = conn,
    cohort_table = f"{results_database_schema}.cohorts_of_interest",
    covariate_settings = covariate_settings)

In [ ]:
covariate_data.summary()

## Creating per-person covariates for a cohort of interest

We can create per-person covariates for one of the cohorts of interest, for example using the default settings:

In [ ]:
covariate_settings = feature_extraction.create_default_covariate_settings()

covariate_data = feature_extraction.get_db_covariate_data(
    connection_details = connection_details,
    cdm_database_schema = cdm_database_schema,
    cohort_database_schema = results_database_schema,
    cohort_table = "cohorts_of_interest",
    cohort_id = 1378382,
    row_id_field = "subject_id", # "subject_id"
    covariate_settings = covariate_settings)

In [ ]:
covariate_data.summary()

### Per-person covariate output format

The main component of the <code>covariateData</code> object is <code>covariates</code>:

In [ ]:
covariate_data.covariates

The columns are defined as follows:

- <code>rowId</code> uniquely identifies a cohort entry. When calling <code>getDbCovariateData</code> we defined <code>rowIdField = "subject_id"</code>, so in this case the <code>rowId</code> is the same as the <code>subject_id</code> in the cohort table. In cases where a single subject can appear in the cohort more than once it is up to the user to create a field in the cohort table that uniquely identifies each cohort entry, and use that as <code>rowIdField</code>.
- <code>covariateId</code> identifies the covariate, and definitions of covariates can be found in the cohortData$covariateRef object.
- <code>covariateValue</code> field provides the value.

## Saving / Loading covariates

Creating covariates can take considerable computing time, and it is probably a good idea to save them for future sessions. Because <code>covariateData</code> objects use <code>Andromeda</code>, we cannot use R’s regular save function. Instead, we’ll have to use the <code>saveCovariateData()</code> function:

In [ ]:
feature_extraction.save_covariate_data(covariate_data, "covariates_py")

We can use the <code>loadCovariateData()</code> function to load the data in a future session:

In [ ]:
covariate_data = feature_extraction.load_covariate_data("covariates_py")

## Removing infrequent covariates, normalizing, and removing redundancy

One reason for generating per-person covariates may be to use them in some form of machine learning. In that case it may be necessary to tidy the data before proceeding. The <code>tidyCovariateData</code> function can perform three tasks:

1. <b>Remove infrequent covariates</b>: Oftentimes the majority of features have non-zero values for only one or a few subjects in the cohort. These features are unlikely to end up in any fitted model, but can increase the computational burden, so removing them could increase performance. By default, covariates appearing in less than .1% of the subjects are removed.
2. <b>Normalization</b>: Scales all covariate values to a value between 0 and 1 (by dividing by the max value for each covariate).
3. <b>Removal of redundancy</b>: If every person in the cohort has the same value for a covariate (e.g. a cohort that is restricted to women will have the same gender covariate value for all) that covariate is redundant. Redundant covariates may pose a problem for some machine learning algorithms, for example causing a simple regression to no longer have a single solution. Similarly, groups of covariates may be redundant (e.g. every person will belong to at least one age group, making one group redundant as it can be defined as the absence of the other groups).

In [ ]:
tidy_covariates = feature_extraction.tidy_covariate_data(
    covariate_data,
    min_fraction = 0.001,
    normalize = True,
    remove_redundancy = True)

If we want to know how many infrequent covariates were removed we can query the <code>metaData</code> object:

In [ ]:
deleted_covariate_ids = tidy_covariates.attr('metaData').get('deletedInfrequentCovariateIds')

Similarly, if we want to know which redundant covariates were removed we can also query the <code>metaData</code> object:

In [ ]:
deleted_covariate_ids = tidy_covariates.attr('metaData').get('deletedRedundantCovariateIds')

If we want to know what these numbers mean, we can use the <code>covariateRef</code> object that is part of any <code>covariateData</code> object. Remember that the covariate data is stored in an <code>Andromeda</code> object, so we should use the proper syntax for querying these objects (see the <code>Andromeda</code> package documentation):

In [ ]:
df = covariate_data.covariate_ref
df[(df.covariateId.isin(deleted_covariate_ids))]

## Filtering covariate data

You can filter the covariate data on row id. See the example below:

In [ ]:
filtered_covariate_data = feature_extraction.filter_by_row_id(covariate_data=covariate_data, row_ids=[2, 68])

# Creating aggregated covariates for a cohort of interest

Often we do not need to have per-person covariates, but instead we are interested in aggregated statistics instead. For example, we may not need to know which persons are male, but would like to know what proportion of the cohort is male. We can aggregate per-person covariates:

In [ ]:
covariate_data2 = feature_extraction.aggregate_covariates(covariate_data)

Of course, if all we wanted was aggregated statistics it would have been more efficient to aggregate them during creation:

In [ ]:
covariate_settings = feature_extraction.create_default_covariate_settings()

covariate_data2 = feature_extraction.get_db_covariate_data(
    connection_details = connection_details,
    cdm_database_schema = cdm_database_schema,
    cohort_database_schema = results_database_schema,
    cohort_table = "cohorts_of_interest",
    cohort_id = 1378382,
    covariate_settings = covariate_settings, 
    aggregated = True)

In [ ]:
covariate_data2.summary()

Note that we specified <code>aggregated = TRUE</code>. Also, we are no longer required to define a <code>rowIdField</code> because we will no longer receive per-person data.

## Aggregated covariate output format

The two main components of the aggregated covariateData object are <code>covariates</code> and <code>covariatesContinuous</code>, for binary and continuous covariates respectively.

The columns of covariates are defined as follows:

- <code>covariateId</code> identifies the covariate, and definitions of covariates can be found in the <code>covariateData$covariateRef</code> object.
- <code>sumValue</code> is the sum of the covariate values. Because these are binary features, this is equivalent to the number of people that have the covariate with a value of 1.
- <code>averageValue</code> is the average covariate value. Because these are binary features, this is equivalent to the proportion of people that have the covariate with a value of 1.

In [ ]:
covariate_data2.covariates

The columns of covariatesContinuous are defined as follows:

- <code>covariateId</code> identifies the covariate, and definitions of covariates can be found in the <code>cohortData$covariateRef</code> object.

- <code>countValue</code> is the number of people that have a value (for continuous variables).
- <code>minValue</code>, <code>maxValue</code>, <code>averageValue</code>, <code>standardDeviation</code>, <code>medianValue</code>, <code>p10Value</code>, <code>p25Value</code>, <code>p75Value</code>, and <code>p90Value</code> all inform on the distribution of covariate values. Note that for some covariates (such as the Charlson comorbidity index) a value of 0 is interpreted as the value 0, while for other covariates (Such as blood pressure) 0 is interpreted as missing, and the distribution statistics are only computed over non-missing values. To learn which continuous covariates fall into which category one can consult the <code>missingMeansZero</code> field in the <code>covariateData$analysisRef</code> object.

In [ ]:
covariate_data2.covariates_continuous

# Creating a table 1

One task supported by the <code>FeatureExtraction</code> package is creating a table of overall study population characteristics that can be include in a paper. Since this is typically the first table in a paper we refer to such a table as ‘table 1’. A default table 1 is available in the <code>FeatureExtraction</code> package:

In [ ]:
result = feature_extraction.create_table1(covariate_data1 = covariate_data2)
result = common.convert_from_r(result)
result.head()

Where applicable, these characteristics are drawn from analyses pertaining the ‘long-term’ windows, so concepts observed in the 365 days before up to and included the cohort start date.

The <code>createTable1</code> function requires a simple specification of what variables to include in the table. The default specifications included in the package can be reviewed by calling the <code>getDefaultTable1Specifications</code> function. The specification reference analysis IDs and covariate IDs, and in the default specification these IDs refer to those in the default covariate settings. It is possible to create custom table 1 specifications and use those instead.

Here we based table 1 on a <code>covariateData</code> object containing all default covariates, even though only a small fraction of covariates are used in the table. If we only want to extract those covariates needed for the table, we can use the <code>createTable1CovariateSettings</code> function:

In [ ]:
covariate_settings = feature_extraction.create_table1_covariate_settings()

covariate_data2b = feature_extraction.get_db_covariate_data(
    connection_details = connection_details,
    cdm_database_schema = cdm_database_schema,
    cohort_database_schema = results_database_schema,
    cohort_table = "cohorts_of_interest",
    cohort_id = 1378382,
    covariate_settings = covariate_settings,
    aggregated = True
)

To load the default Table1 specifications you can run:

In [ ]:
default_table1_specs = feature_extraction.get_default_table1_specifications()
default_table1_specs = common.convert_from_r(default_table1_specs)
default_table1_specs

# Comparing two cohorts

Another task supported by the <code>FeatureExtraction</code> package is comparing two cohorts of interest. Suppose we want to compare two cohorts only on the variables included in the default table 1:

In [ ]:
settings = feature_extraction.create_table1_covariate_settings(
    excluded_covariate_concept_ids = [1378382, 1519936],
    add_descendants_to_exclude = True
)

cov_paclitaxel = feature_extraction.get_db_covariate_data(
    connection_details = connection_details,
    cdm_database_schema = cdm_database_schema,
    cohort_database_schema = results_database_schema,
    cohort_table = "cohorts_of_interest",
    cohort_id = 1378382,
    covariate_settings = settings,
    aggregated = True
)

cov_etonogestrel = feature_extraction.get_db_covariate_data(
    connection_details = connection_details,
    cdm_database_schema = cdm_database_schema,
    cohort_database_schema = results_database_schema,
    cohort_table = "cohorts_of_interest",
    cohort_id = 1519936,
    covariate_settings = settings,
    aggregated = True
)

In [ ]:
std = feature_extraction.compute_standardized_difference(cov_paclitaxel, cov_etonogestrel)
std = common.convert_from_r(std)

In this example we have chosen to exclude any covariates derived from the two concepts that were used to define the two cohorts: etonogestrel (1378382), and paclitaxel (1519936). We compute the standardized difference between the remaining covariates.

In [ ]:
std.head()

The <code>stdDiff</code> column contains the standardized difference. By default the data is ranked in descending order of the absolute value of the standardized difference, showing the covariate with the largest difference first.

We can also show the comparison as a standard table 1:

In [ ]:
result = feature_extraction.create_table1(
    covariate_data1 = cov_paclitaxel,
    covariate_data2 = cov_etonogestrel,
    output = "two columns")

result = common.convert_from_r(result)
result.head()

# Disconnect

In [ ]:
Connect.disconnect(conn)

print('Done')